In [1]:
input_folder = r'\\data.ebb.gatech.edu\research\Kemp\Andrew\102223_doxinduction\zstack'
output_folder = r'\\data.ebb.gatech.edu\research\Kemp\Andrew\102223_doxinduction\result'
xlsx_output_path = r'\\data.ebb.gatech.edu\research\Kemp\Andrew\102223_doxinduction\result\intensity_bgc.xlsx'

dapi_position = 0 # position of dapi channel in the image, start to count from 0
bits = 16 # change to 8 is your image intensity maximum is 255
channels_of_interest = [1,2,3] # list of positions of channels to analyse, count from 0
#channels_names = ["cy5","cy5_bgc","dapi","dapi_bgc","gfp","gfp_bgc","rfp","rfp_bgc"]
#channels_names = ["dapi","gfp","rfp","dapi_bgc","gfp_bgc","rfp_bgc"]
channels_names = ["dapi", "FOXA2", "GATA6", "FOXF1"]
 ### ^ This needs to match the channels of the nikon image stack.
dpi = 600 # output image quality

segmentation_tile_size = 101
substract_background = 0

In [2]:
# import libraries
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
import xlsxwriter
from skimage.measure import regionprops
from skimage.filters import gaussian, threshold_local
from skimage.feature import peak_local_max
from skimage.segmentation import watershed
from skimage.morphology import label
from scipy import ndimage
from matplotlib import colors
from PIL import Image
import time

def segmentation(image,RadThr,Thr):
    DistanceBlurRad = 0.5
    # crop initial DAPI confocal
    h_crop,w_crop = image.shape
    dapi_thresh = threshold_local(image, 3501)
    dapi_mask = image>dapi_thresh
    mask = threshold_local(image, RadThr, offset = -Thr)
    mask = image>mask
    mask = dapi_mask&mask
    distance = ndimage.distance_transform_edt(mask)
    distance = gaussian(distance, sigma=DistanceBlurRad)
    #local_maxi = peak_local_max(distance, exclude_border=False, labels=mask)
    local_maxi = peak_local_max(distance, indices=False,exclude_border=False, labels=mask)
    markers = label(local_maxi)
    nuclei_labels = watershed(-distance, markers, mask=mask)
    cytoprops = regionprops(nuclei_labels)
    return cytoprops

def cleanup(cellprops):
    n = len(cellprops)
    area = np.zeros(n)
    for i in range(n):
        area[i] = cellprops[i]["area"]
    area_std, area_mean = np.std(area), np.mean(area)
    cells, X, Y, area, perimeter, eccentricity = [],[],[],[],[],[]
    for i in range(n):
        if cellprops[i]['area'] < area_mean+3*area_std and cellprops[i]['area'] > max(5,area_mean-3*area_std):
            cells.append(cellprops[i])
            X.append(cellprops[i]['centroid'][0])
            Y.append(cellprops[i]['centroid'][1])
            area.append(cellprops[i]["area"])
            perimeter.append(cellprops[i]["perimeter"])
            eccentricity.append(cellprops[i]["eccentricity"])
    return cells, np.array(X), np.array(Y), area, perimeter, eccentricity

def overlay_cells(image, cells):
    h,w = image.shape
    output = np.zeros((h,w))
    n = len(cells)
    intensities = np.zeros(n)
    for i in range(n):
        for coord in cells[i]["coords"]:
            intensities[i] += image[coord[0],coord[1]]
        intensities[i] = intensities[i]/cells[i]["area"]
        for coord in cells[i]["coords"]:
            output[coord[0],coord[1]] = intensities[i]
    return intensities,output

def plot(image,name,dpi,max=0):
    if max == 0:
      m = np.max(image)
    else:
      m = max
    cmap = colors.ListedColormap(['white','darkblue','blue','cornflowerblue',
                                  'cyan','aquamarine','lime','greenyellow','yellow','gold','orange','red','brown'])
    bounds=[0,1e-9,m/12,m/6,m/4,m/3,m/2.4,m/2,0.58*m,m/1.5,0.75*m,m/1.2,0.91*m,m]
    norm = colors.BoundaryNorm(bounds, cmap.N)
    image = plt.imshow(image, cmap=cmap, norm=norm)
    plt.colorbar(image, cmap='jet')
    plt.savefig(name,dpi=dpi,bbox_inches = "tight")
    plt.close()


In [3]:
import tifffile
import pandas as pd

img_list = os.listdir(input_folder)
if not os.path.exists(output_folder):os.mkdir(output_folder)
for path in img_list:
  start = time.time()
  img = tifffile.imread(rf'{input_folder}\{path}')
  ch_img = img[dapi_position,:,:]
  dapi = np.array(ch_img)
  print(path)
  print(dapi.shape)
  dapi_segmentation = segmentation(dapi,segmentation_tile_size,substract_background)
  dapi_segmentation, X, Y, area, perimeter, eccentricity = cleanup(dapi_segmentation)
  header = ["X","Y","area", "perimeter", "eccentricity"] + channels_names
  data = [X,Y,area, perimeter, eccentricity]
  if not os.path.exists(output_folder+"/"+path):os.mkdir(output_folder+"/"+path)
  overlay = overlay_cells(dapi, dapi_segmentation)
  plot(overlay[1],output_folder+"/"+path+"/dapi.png",dpi,max=2**bits)
  data.append(overlay[0])
  for ch in channels_of_interest:
    #img.seek(ch)
    ch_img = img[ch,:,:]
    overlay = overlay_cells(np.array(ch_img), dapi_segmentation)
    plot(overlay[1],output_folder+"/"+path+"/"+channels_names[ch]+".png",dpi)
    data.append(overlay[0])
  df_data = np.asarray(data)
  df = pd.DataFrame(df_data.T, columns = header)
  df.to_csv(rf'{output_folder}\{path}\data.csv', index=False)
  #img.close()
  print("time taken:",time.time()-start)

d5_FOXA2_GATA6_FOXF1_0dox_20x_0.tif
(6221, 6221)


In [10]:
import os
import tifffile

#### This code splits n x m tiff stacks into m stacks of n channel images

input_folder = r'\\data.ebb.gatech.edu\research\Kemp\Andrew\102223_doxinduction\tiff'
output_folder = r'\\data.ebb.gatech.edu\research\Kemp\Andrew\102223_doxinduction\zstack'
img_list = os.listdir(input_folder)
if not os.path.exists(output_folder):
    os.mkdir(output_folder)
count = 0
for path in img_list:
    im = tifffile.imread(rf'{input_folder}\{path}')
    print(path)
    print(im.shape)
    for slice in range(im.shape[0]):
        print(f'Slice {slice}...')
        tifffile.imwrite(rf'{output_folder}\{path[:-4]}_{slice}.tif', im[slice,:,:,:])


d5_FOXA2_GATA6_FOXF1_0dox_20x.tif
(22, 4, 6221, 6221)
Slice 0...
Slice 1...
Slice 2...
Slice 3...
Slice 4...
Slice 5...
Slice 6...
Slice 7...
Slice 8...
Slice 9...
Slice 10...
Slice 11...
Slice 12...
Slice 13...
Slice 14...
Slice 15...
Slice 16...
Slice 17...
Slice 18...
Slice 19...
Slice 20...
Slice 21...
d5_FOXA2_GATA6_FOXF1_0dox_20x_2.tif
(21, 4, 6221, 6221)
Slice 0...
Slice 1...
Slice 2...
Slice 3...
Slice 4...
Slice 5...
Slice 6...
Slice 7...
Slice 8...
Slice 9...
Slice 10...
Slice 11...
Slice 12...
Slice 13...
Slice 14...
Slice 15...
Slice 16...
Slice 17...
Slice 18...
Slice 19...
Slice 20...
d5_FOXA2_GATA6_FOXF1_0dox_20x_3.tif
(33, 4, 6221, 6221)
Slice 0...
Slice 1...
Slice 2...
Slice 3...
Slice 4...
Slice 5...
Slice 6...
Slice 7...
Slice 8...
Slice 9...
Slice 10...
Slice 11...
Slice 12...
Slice 13...
Slice 14...
Slice 15...
Slice 16...
Slice 17...
Slice 18...
Slice 19...
Slice 20...
Slice 21...
Slice 22...
Slice 23...
Slice 24...
Slice 25...
Slice 26...
Slice 27...
Slice 28...


In [31]:
import pandas as pd


excel_header = ["X","Y","area", "perimeter", "eccentricity"]
data = []
for i in range(len(excel_header)):
    data.append([i] * 100)
np_data = np.asarray(data)
df = pd.DataFrame(np_data.T, columns=excel_header)
print(df)

    X  Y  area  perimeter  eccentricity
0   0  1     2          3             4
1   0  1     2          3             4
2   0  1     2          3             4
3   0  1     2          3             4
4   0  1     2          3             4
.. .. ..   ...        ...           ...
95  0  1     2          3             4
96  0  1     2          3             4
97  0  1     2          3             4
98  0  1     2          3             4
99  0  1     2          3             4

[100 rows x 5 columns]
